<a href="https://colab.research.google.com/github/BouchekirRedouane/TestMNIST_LiRPA/blob/main/testLiRPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uninstall existing Pytorch on Colab, which might be incompatible or buggy.
!pip uninstall --yes torch torchvision torchaudio torchtext
# Install Pytorch 1.8.2 LTS (Long Term Support) and auto_LiRPA. It might take a few minutes depending on network speed.
!pip install torch==1.8.2+cu102 torchvision==0.9.2+cu102 torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install git+https://github.com/KaidiXu/auto_LiRPA.git
# Clear installation output to avoid clutter.
from IPython.display import clear_output
clear_output()

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from __future__ import print_function

import argparse
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision

In [3]:
from auto_LiRPA import BoundedModule, BoundedTensor
from auto_LiRPA.perturbations import *

In [4]:
## Step 1: Define computational graph by implementing forward()
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

In [5]:
# This simple model comes from https://github.com/locuslab/convex_adversarial
def mnist_model():
    model = nn.Sequential(
        nn.Conv2d(1, 16, 4, stride=2, padding=1),
        nn.ReLU(),
        nn.Conv2d(16, 32, 4, stride=2, padding=1),
        nn.ReLU(),
        Flatten(),
        nn.Linear(32*7*7,100),
        nn.ReLU(),
        nn.Linear(100, 10)
    )
    return model

model = mnist_model()

In [9]:
## Step 2: Prepare dataset as usual
test_data = torchvision.datasets.MNIST("./data", train=False, download=True, transform=torchvision.transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [10]:
# For illustration we only use 2 image from dataset
N = 2
n_classes = 10
image = test_data.data[:N].view(N,1,28,28)
true_label = test_data.targets[:N]

In [12]:
# Convert to float
image = image.to(torch.float32) / 255.0
if torch.cuda.is_available():
    image = image.cuda()
    model = model.cuda()

In [13]:
## Step 3: wrap model with auto_LiRPA
# The second parameter is for constructing the trace of the computational graph, and its content is not important.
lirpa_model = BoundedModule(model, torch.empty_like(image), device=image.device)
print('Running on', image.device)

Running on cpu


In [46]:
## Step 4: Compute bounds using LiRPA given a perturbation
eps = 0.3
norm = float("inf")
ptb = PerturbationLpNorm(norm = norm, eps = eps)
image = BoundedTensor(image, ptb)
# Get model prediction as usual
pred = lirpa_model(image)
label = torch.argmax(pred, dim=1).cpu().detach().numpy()
print(pred)
print(label)

tensor([[ 0.0788, -0.0588,  0.0054,  0.0024, -0.0820, -0.0665, -0.0918, -0.0711,
          0.0762, -0.0882],
        [ 0.0859, -0.0364,  0.0028,  0.0080, -0.0768, -0.0672, -0.1036, -0.0688,
          0.0562, -0.0857]], grad_fn=<AddBackward0>)
[0 0]


In [15]:
## Step 5: Compute bounds for final output
for method in ['IBP', 'IBP+backward (CROWN-IBP)', 'backward (CROWN)', 'CROWN-Optimized (alpha-CROWN)']:
# for method in ['IBP', 'IBP+backward (CROWN-IBP)', 'backward (CROWN)', ]:
    print("Bounding method:", method)
    if 'Optimized' in method:
        # For optimized bound, you can change the number of iterations, learning rate, etc here. Also you can increase verbosity to see per-iteration loss values.
        lirpa_model.set_bound_opts({'optimize_bound_args': {'ob_iteration': 20, 'ob_lr': 0.1, 'ob_verbose': 0}})
    lb, ub = lirpa_model.compute_bounds(x=(image,), method=method.split()[0])
    for i in range(N):
        print("Image {} top-1 prediction {} ground-truth {}".format(i, label[i], true_label[i]))
        for j in range(n_classes):
            indicator = '(ground-truth)' if j == true_label[i] else ''
            print("f_{j}(x_0): {l:8.3f} <= f_{j}(x_0+delta) <= {u:8.3f} {ind}".format(
                j=j, l=lb[i][j].item(), u=ub[i][j].item(), ind=indicator))
    print()

Bounding method: IBP
Image 0 top-1 prediction 0 ground-truth 7
f_0(x_0):  -40.915 <= f_0(x_0+delta) <=   45.493 
f_1(x_0):  -55.083 <= f_1(x_0+delta) <=   44.207 
f_2(x_0):  -47.222 <= f_2(x_0+delta) <=   43.427 
f_3(x_0):  -55.394 <= f_3(x_0+delta) <=   46.114 
f_4(x_0):  -40.961 <= f_4(x_0+delta) <=   57.636 
f_5(x_0):  -46.640 <= f_5(x_0+delta) <=   50.198 
f_6(x_0):  -47.297 <= f_6(x_0+delta) <=   44.373 
f_7(x_0):  -38.951 <= f_7(x_0+delta) <=   49.877 (ground-truth)
f_8(x_0):  -57.443 <= f_8(x_0+delta) <=   40.695 
f_9(x_0):  -44.606 <= f_9(x_0+delta) <=   49.373 
Image 1 top-1 prediction 0 ground-truth 2
f_0(x_0):  -41.091 <= f_0(x_0+delta) <=   45.731 
f_1(x_0):  -55.328 <= f_1(x_0+delta) <=   44.432 
f_2(x_0):  -47.458 <= f_2(x_0+delta) <=   43.633 (ground-truth)
f_3(x_0):  -55.661 <= f_3(x_0+delta) <=   46.318 
f_4(x_0):  -41.151 <= f_4(x_0+delta) <=   57.930 
f_5(x_0):  -46.851 <= f_5(x_0+delta) <=   50.440 
f_6(x_0):  -47.532 <= f_6(x_0+delta) <=   44.585 
f_7(x_0):  -39.13

In [16]:
## An example for computing margin bounds.
# In compute_bounds() function you can pass in a specification matrix C, which is a final linear matrix applied to the last layer NN output.
# For example, if you are interested in the margin between the groundtruth class and another class, you can use C to specify the margin.
# This generally yields tighter bounds.
# Here we compute the margin between groundtruth class and groundtruth class + 1.
# If you have more than 1 specifications per batch element, you can expand the second dimension of C (it is 1 here for demonstration).
lirpa_model = BoundedModule(model, torch.empty_like(image), device=image.device)
C = torch.zeros(size=(N, 1, n_classes), device=image.device)
groundtruth = true_label.to(device=image.device).unsqueeze(1).unsqueeze(1)
target_label = (groundtruth + 1) % n_classes
C.scatter_(dim=2, index=groundtruth, value=1.0)
C.scatter_(dim=2, index=target_label, value=-1.0)
print('Computing bounds with a specification matrix:\n', C)

for method in ['IBP', 'IBP+backward (CROWN-IBP)', 'backward (CROWN)', 'CROWN-Optimized (alpha-CROWN)']:
    print("Bounding method:", method)
    if 'Optimized' in method:
        # For optimized bound, you can change the number of iterations, learning rate, etc here. Also you can increase verbosity to see per-iteration loss values.
        lirpa_model.set_bound_opts({'optimize_bound_args': {'ob_iteration': 20, 'ob_lr': 0.1, 'ob_verbose': 0}})
    lb, ub = lirpa_model.compute_bounds(x=(image,), method=method.split()[0], C=C)
    for i in range(N):
        print("Image {} top-1 prediction {} ground-truth {}".format(i, label[i], true_label[i]))
        print("margin bounds: {l:8.3f} <= f_{j}(x_0+delta) - f_{target}(x_0+delta) <= {u:8.3f}".format(
            j=true_label[i], target=(true_label[i] + 1) % n_classes, l=lb[i][0].item(), u=ub[i][0].item()))
    print()

Computing bounds with a specification matrix:
 tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  0.]],

        [[ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.]]])
Bounding method: IBP
Image 0 top-1 prediction 0 ground-truth 7
margin bounds:  -46.789 <= f_7(x_0+delta) - f_8(x_0+delta) <=   74.464
Image 1 top-1 prediction 0 ground-truth 2
margin bounds:  -62.960 <= f_2(x_0+delta) - f_3(x_0+delta) <=   68.477

Bounding method: IBP+backward (CROWN-IBP)
Image 0 top-1 prediction 0 ground-truth 7
margin bounds:  -27.868 <= f_7(x_0+delta) - f_8(x_0+delta) <=   41.694
Image 1 top-1 prediction 0 ground-truth 2
margin bounds:  -36.372 <= f_2(x_0+delta) - f_3(x_0+delta) <=   38.283

Bounding method: backward (CROWN)
Image 0 top-1 prediction 0 ground-truth 7
margin bounds:   -9.848 <= f_7(x_0+delta) - f_8(x_0+delta) <=   13.708
Image 1 top-1 prediction 0 ground-truth 2
margin bounds:  -11.922 <= f_2(x_0+delta) - f_3(x_0+delta) <=   12.452

Bounding method: CROWN-Optimized (alpha-CROWN)
al